### 필수과제1. 임계값 조절하여 분류 테스트 변화 확인해  보기
- 임계값을 3개정도 (원하는 값으로) 잡고 각각의 평가 지표 변화 확인해 보기!

In [27]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, precision_score,recall_score,f1_score
from sklearn.metrics import precision_recall_curve 
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Binarizer

import numpy as np

In [21]:
import seaborn as sns

titanic=sns.load_dataset('titanic')
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [33]:
def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    print('오차 행렬')
    print(confusion)
    print(' 정밀도: {0:.4f}, 재현율: {1:.4f}'.format(precision ,recall))

In [32]:
tt_x=titanic[['pclass','sibsp','parch','fare']]
tt_y=titanic[['survived']]

lr_clf = LogisticRegression()

lr_clf.fit(tt_x , tt_y)
pred = lr_clf.predict(tt_x)
get_clf_eval(tt_y , pred)

오차 행렬
[[461  88]
 [190 152]]
 정밀도: 0.6333, 재현율: 0.4444


c:\Users\KYW\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
pred_proba = lr_clf.predict_proba(tt_x)
pred  = lr_clf.predict(tt_x)
print('pred_proba()결과 Shape : {0}'.format(pred_proba.shape))
print('pred_proba array에서 앞 3개만 샘플로 추출 \n:', pred_proba[:3])

# 예측 확률 array 와 예측 결과값 array 를 concatenate 하여 예측 확률과 결과값을 한눈에 확인
pred_proba_result = np.concatenate([pred_proba , pred.reshape(-1,1)],axis=1)
print('두개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n',pred_proba_result[:3])

pred_proba()결과 Shape : (891, 2)
pred_proba array에서 앞 3개만 샘플로 추출 
: [[0.78213545 0.21786455]
 [0.39477788 0.60522212]
 [0.75559028 0.24440972]]
두개의 class 중에서 더 큰 확률을 클래스 값으로 예측 
 [[0.78213545 0.21786455 0.        ]
 [0.39477788 0.60522212 1.        ]
 [0.75559028 0.24440972 0.        ]]


In [18]:
from sklearn.preprocessing import Binarizer

X = [[ 1, -1,  2],
     [ 2,  0,  0],
     [ 0,  1.1, 1.2]]

# threshold 기준값보다 같거나 작으면 0을, 크면 1을 반환
binarizer = Binarizer(threshold=1.1)                     
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [31]:
for threshold in [0.3,0.5,0.7,0.9]:
    print('--------임계값:',threshold,'--------')
    pred_proba_1 = pred_proba[:,1].reshape(-1,1)
    binarizer = Binarizer(threshold=threshold).fit(pred_proba_1) 
    custom_predict = binarizer.transform(pred_proba_1)
    get_clf_eval(tt_y, custom_predict)

# predict_proba( ) 반환값의 두번째 컬럼 , 즉 Positive 클래스 컬럼 하나만 추출하여 Binarizer를 적용


--------임계값: 0.3 --------
오차 행렬
[[349 200]
 [109 233]]
 정밀도: 0.5381, 재현율: 0.6813
--------임계값: 0.5 --------
오차 행렬
[[461  88]
 [190 152]]
 정밀도: 0.6333, 재현율: 0.4444
--------임계값: 0.7 --------
오차 행렬
[[535  14]
 [306  36]]
 정밀도: 0.7200, 재현율: 0.1053
--------임계값: 0.9 --------
오차 행렬
[[548   1]
 [339   3]]
 정밀도: 0.7500, 재현율: 0.0088
